In [38]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
filename = "uklon_sample_3_months.csv"

In [3]:
parse_dates = ['created_at']

In [4]:
df = pd.read_csv(filename, delimiter=";", decimal=",", parse_dates=parse_dates)

/home/temenid/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,order_id,ride_id,created_at,pickup_lat,pickup_lon,dropoff_lat,dropoff_lon,user_id,driver_id,ride_distance,...,ooExtraCost,ooInitialCost,ooIdleCost,oapAzureMlCost,oapAzureMlCostBid,oapCostProvider,ooSurgeMultiplier,driver_lat,driver_lon,dtTaxiDriver_tdID
0,88314768,764A34A6-5C92-4A0C-90E8-82BDA5A5F9C1,2017-10-31 23:59:59.143,50.44890594,30.38644790,50.41026687,30.41221618,420361.0,568114.0,9.10,...,1.0,115.0,0.0,114.0,1.0,AzureML,1.35,50.43701166,30.38368000,568114.0
1,88314766,299E8328-8BE6-4097-AF81-4834478B4D51,2017-10-31 23:59:58.577,50.44600000,30.57620000,50.41460418,30.38434982,722883.0,553135.0,19.74,...,0.0,146.0,0.0,146.0,0.0,AzureML,1.00,50.44951166,30.59553666,553135.0
2,88314765,7AABF746-29E9-47FF-ACE6-20CB27C8D658,2017-10-31 23:59:57.883,50.43580000,30.48970000,50.51310348,30.43841171,NaN,NaN,11.16,...,0.0,98.0,0.0,117.0,1.0,Uklon,1.00,NaN,NaN,NaN
3,88314764,CDB5BFDF-BDF9-40A5-A86A-71651F350242,2017-10-31 23:59:56.403,50.43912506,30.51979827,50.44469833,30.51986503,54185.0,NaN,11.07,...,-13.0,100.0,0.0,113.0,1.0,NaN,1.15,NaN,NaN,NaN
4,88314763,4EDC7FB9-5E34-4132-A2B3-B926F81DB439,2017-10-31 23:59:55.050,50.43861007,30.45953559,50.43933868,30.44442749,693323.0,568920.0,11.08,...,-7.0,100.0,0.0,107.0,1.0,AzureML,1.09,50.42497166,30.45896500,568920.0


In [6]:
df.columns

Index(['order_id', 'ride_id', 'created_at', 'pickup_lat', 'pickup_lon',
       'dropoff_lat', 'dropoff_lon', 'user_id', 'driver_id', 'ride_distance',
       'ooCancelReason', 'ooSuburbanDistance', 'ooCarSearchTime', 'ooCost',
       'ooExtraCost', 'ooInitialCost', 'ooIdleCost', 'oapAzureMlCost',
       'oapAzureMlCostBid', 'oapCostProvider', 'ooSurgeMultiplier',
       'driver_lat', 'driver_lon', 'dtTaxiDriver_tdID'],
      dtype='object')

In [7]:
df.isnull().sum()

order_id                    0
ride_id                     0
created_at                  0
pickup_lat               1326
pickup_lon               1326
dropoff_lat              1710
dropoff_lon              1710
user_id                812181
driver_id             3629128
ride_distance               0
ooCancelReason              0
ooSuburbanDistance          0
ooCarSearchTime             0
ooCost                      0
ooExtraCost                 0
ooInitialCost               0
ooIdleCost                  0
oapAzureMlCost        3696104
oapAzureMlCostBid     3696104
oapCostProvider       4176766
ooSurgeMultiplier           0
driver_lat            4046614
driver_lon            4046614
dtTaxiDriver_tdID     4046614
dtype: int64

In [8]:
df.shape

(6444521, 24)

In [9]:
# Remove rows with missing values in column 'driver_id'. The order was not accepted. there was no trip
new_df = df.dropna(subset=['driver_id'],inplace=False)

In [18]:
# leave only uncanceled ride
new_df = new_df.loc[df['ooCancelReason'] == 0]

In [19]:
new_df.shape

(2371694, 24)

In [45]:
# initialy select the following columns
data = new_df[['created_at','ride_distance', 'ooSuburbanDistance', 'ooCarSearchTime', 'ooCost']]

In [46]:
data.head()

,created_at,ride_distance,ooSuburbanDistance,ooCarSearchTime,ooCost
0,2017-10-31 23:59:59.143,9.10,0,13,115.0
1,2017-10-31 23:59:58.577,19.74,0,7,146.0
4,2017-10-31 23:59:55.050,11.08,0,1,100.0
5,2017-10-31 23:59:54.420,9.24,0,69,88.0
7,2017-10-31 23:59:50.053,3.73,0,4,55.0


In [47]:
# count the NaN values in a columns
data.isnull().sum()

created_at            0
ride_distance         0
ooSuburbanDistance    0
ooCarSearchTime       0
ooCost                0
dtype: int64

In [48]:
# some simple feature engineering with date
# as categorical
data['quarter'] = data['created_at'].dt.quarter.astype(str)
data['month'] = data['created_at'].dt.month.astype(str)
data['day'] = data['created_at'].dt.day.astype(str)
data['dayofweek'] = data['created_at'].dt.dayofweek.astype(str)

/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [49]:
# as categorical
data['quarter_int'] = data['created_at'].dt.quarter.astype(int)
data['month_int'] = data['created_at'].dt.month.astype(int)
data['day_int'] = data['created_at'].dt.day.astype(int)
data['dayofweek_int'] = data['created_at'].dt.dayofweek.astype(int)

/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/temenid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [50]:
data.head()

,created_at,ride_distance,ooSuburbanDistance,ooCarSearchTime,ooCost,quarter,month,day,dayofweek,quarter_int,month_int,day_int,dayofweek_int
0,2017-10-31 23:59:59.143,9.10,0,13,115.0,4,10,31,1,4,10,31,1
1,2017-10-31 23:59:58.577,19.74,0,7,146.0,4,10,31,1,4,10,31,1
4,2017-10-31 23:59:55.050,11.08,0,1,100.0,4,10,31,1,4,10,31,1
5,2017-10-31 23:59:54.420,9.24,0,69,88.0,4,10,31,1,4,10,31,1
7,2017-10-31 23:59:50.053,3.73,0,4,55.0,4,10,31,1,4,10,31,1


In [51]:
data.shape

(2371694, 13)

In [52]:
data.describe()

,ride_distance,ooSuburbanDistance,ooCarSearchTime,ooCost,quarter_int,month_int,day_int,dayofweek_int
count,2.371694e+06,2371694.0,2.371694e+06,2.371694e+06,2.371694e+06,2.371694e+06,2.371694e+06,2.371694e+06
mean,1.104782e+01,0.0,1.909313e+01,1.053575e+02,3.361107e+00,9.048252e+00,1.585837e+01,3.008878e+00
std,7.240361e+00,0.0,3.114566e+01,5.001793e+01,4.803217e-01,8.195334e-01,8.864580e+00,1.954233e+00
min,0.000000e+00,0.0,-3.528000e+03,3.000000e+01,3.000000e+00,8.000000e+00,1.000000e+00,0.000000e+00
25%,5.610000e+00,0.0,5.000000e+00,7.000000e+01,3.000000e+00,8.000000e+00,8.000000e+00,1.000000e+00
50%,9.960000e+00,0.0,9.000000e+00,1.000000e+02,3.000000e+00,9.000000e+00,1.600000e+01,3.000000e+00
75%,1.451000e+01,0.0,2.300000e+01,1.250000e+02,4.000000e+00,1.000000e+01,2.300000e+01,5.000000e+00
max,1.443100e+02,0.0,4.747000e+03,4.062000e+03,4.000000e+00,1.000000e+01,3.100000e+01,6.000000e+00


In [53]:
# In our dataset, we have both categorical and numeric data. select them for different processing

In [54]:
# CATEGORICAL (for the moment only these)
catFeatures = ['quarter','month','day','dayofweek']

In [55]:
# Convert categorical variable into dummy/indicator variables
df_cat = pd.get_dummies(data[catFeatures])

In [56]:
# let's check
df_cat.head()

,quarter_3,quarter_4,month_10,month_8,month_9,day_1,day_10,day_11,day_12,day_13,...,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
7,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [57]:
# concat dummy/indicator variables to dataframe
data = pd.concat([data, df_cat], axis=1)

In [58]:
data.head()

,created_at,ride_distance,ooSuburbanDistance,ooCarSearchTime,ooCost,quarter,month,day,dayofweek,quarter_int,...,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,2017-10-31 23:59:59.143,9.10,0,13,115.0,4,10,31,1,4,...,0,0,0,0,1,0,0,0,0,0
1,2017-10-31 23:59:58.577,19.74,0,7,146.0,4,10,31,1,4,...,0,0,0,0,1,0,0,0,0,0
4,2017-10-31 23:59:55.050,11.08,0,1,100.0,4,10,31,1,4,...,0,0,0,0,1,0,0,0,0,0
5,2017-10-31 23:59:54.420,9.24,0,69,88.0,4,10,31,1,4,...,0,0,0,0,1,0,0,0,0,0
7,2017-10-31 23:59:50.053,3.73,0,4,55.0,4,10,31,1,4,...,0,0,0,0,1,0,0,0,0,0


In [59]:
# drop useless categorical column (because we have dummy/indicator variables)
data.drop(catFeatures, axis=1,inplace=True)

In [60]:
data.head()

,created_at,ride_distance,ooSuburbanDistance,ooCarSearchTime,ooCost,quarter_int,month_int,day_int,dayofweek_int,quarter_3,...,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,2017-10-31 23:59:59.143,9.10,0,13,115.0,4,10,31,1,0,...,0,0,0,0,1,0,0,0,0,0
1,2017-10-31 23:59:58.577,19.74,0,7,146.0,4,10,31,1,0,...,0,0,0,0,1,0,0,0,0,0
4,2017-10-31 23:59:55.050,11.08,0,1,100.0,4,10,31,1,0,...,0,0,0,0,1,0,0,0,0,0
5,2017-10-31 23:59:54.420,9.24,0,69,88.0,4,10,31,1,0,...,0,0,0,0,1,0,0,0,0,0
7,2017-10-31 23:59:50.053,3.73,0,4,55.0,4,10,31,1,0,...,0,0,0,0,1,0,0,0,0,0


In [61]:
#INTEGER
intFeatures = ['ride_distance','ooSuburbanDistance','ooCarSearchTime','quarter_int','month_int','day_int','dayofweek_int']

In [62]:
# integer features scaling
scaler = MinMaxScaler()
data[intFeatures] = scaler.fit_transform(data[intFeatures])

In [63]:
# split features and target
dfX = data.drop(['created_at','ooCost'], axis=1)
dfY = data['ooCost']

In [64]:
dfX.head()

,ride_distance,ooSuburbanDistance,ooCarSearchTime,quarter_int,month_int,day_int,dayofweek_int,quarter_3,quarter_4,month_10,...,day_7,day_8,day_9,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,0.063059,0.0,0.427915,1.0,1.0,1.0,0.166667,0,1,1,...,0,0,0,0,1,0,0,0,0,0
1,0.136789,0.0,0.427190,1.0,1.0,1.0,0.166667,0,1,1,...,0,0,0,0,1,0,0,0,0,0
4,0.076779,0.0,0.426465,1.0,1.0,1.0,0.166667,0,1,1,...,0,0,0,0,1,0,0,0,0,0
5,0.064029,0.0,0.434683,1.0,1.0,1.0,0.166667,0,1,1,...,0,0,0,0,1,0,0,0,0,0
7,0.025847,0.0,0.426828,1.0,1.0,1.0,0.166667,0,1,1,...,0,0,0,0,1,0,0,0,0,0


In [65]:
dfY.head()

0    115.0
1    146.0
4    100.0
5     88.0
7     55.0
Name: ooCost, dtype: float64